In [ ]:
import cProfile, pstats, StringIO
import pycuda.gpuarray as gpuarray
from pycuda.autoinit import context
import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np

In [2]:
from pycuda.compiler import SourceModule

with open('strain_device.cu','r') as cudaSrc:
    src=cudaSrc.read()
mod = SourceModule(src)
sim_func=mod.get_function('Simulate_for_Strain')
hit_func=mod.get_function('Hit_Score')
tExref = mod.get_texref("tcExpData")
tGref= mod.get_texref('tfG')

In [1]:
# variables for that specific grain and detector, in global namespace
from InitStrain import Det1, g15Gs1_2nd, g15Info1_2nd, g15pos2nd, exp, g15orien2nd
# Number of G vectors
NumG=96
## Lim for window position
Lim=[]
for ii in range(96):
    Lim.append(np.load('/home/fyshen13/Strain/g15Ps1_2nd_bf/limit{0:d}.npy'.format(ii))[0])
Lim=np.array(Lim).astype(np.int32)
LimD=gpuarray.to_gpu(Lim)
## whichOmega for choosing between omega1 and omega2
whichOmega=np.zeros(len(Lim),dtype=np.int32)
for ii in range(len(Lim)):
    if g15Info1_2nd[ii]['WhichOmega']=='b':
        whichOmega[ii]=2
    else:
        whichOmega[ii]=1
whichOmegaD=gpuarray.to_gpu(whichOmega)
# MaxInt for normalize the weight of each spot 
#(because different spots have very different intensity but we want them equal weighted) 
MaxInt=np.load('MaxInt_g15Ps1_2nd.npy')
MaxIntD=gpuarray.to_gpu(MaxInt.astype(np.float32))

# Det parameters
afDetInfoH=np.concatenate([[2048,2048,0.001454,0.001454],
                           Det1.CoordOrigin,Det1.Norm,Det1.Jvector,Det1.Kvector]).astype(np.float32)
afDetInfoD=gpuarray.to_gpu(afDetInfoH)

In [5]:
#transfer the ExpImgs and all Gs to texture memory
def initTEX():
    AllIm=np.zeros(shape=(160,300,96*45),dtype=np.uint32,order='F')
    for ii in range(96):
        tmp=np.load('/home/fyshen13/Strain/g15Ps1_2nd_filtered/Im{0:d}.npy'.format(ii))
        tmp=np.moveaxis(tmp, 0, 2)
        AllIm[:tmp.shape[0],:tmp.shape[1],ii*45:(ii+1)*45]=tmp
    shape=AllIm.shape
    descr = cuda.ArrayDescriptor3D()
    descr.width = shape[0]
    descr.height = shape[1]
    descr.depth = shape[2]
    descr.format = cuda.dtype_to_array_format(AllIm.dtype)
    descr.num_channels = 1
    descr.flags = 0
    ary = cuda.Array(descr)
    copy = cuda.Memcpy3D()
    copy.set_src_host(AllIm)
    copy.set_dst_array(ary)
    copy.width_in_bytes = copy.src_pitch = AllIm.strides[1]
    copy.src_height = copy.height = shape[1]
    copy.depth = shape[2]
    copy()
    tExref.set_array(ary)
    
    tGref.set_array(cuda.matrix_to_array(np.transpose(g15Gs1_2nd).astype(np.float32),order='F'))

In [27]:
def CrossEntropyMethod(x,y,NumD=10000,numCut=100,MaxIter=100,S_init=np.eye(3)):

    S=np.random.multivariate_normal(
        np.zeros(9),np.eye(9)*1e-4,size=(NumD)).reshape((NumD,3,3),order='C')+np.tile(S_init,(NumD,1,1))
    

    SD=gpuarray.to_gpu(S.ravel().astype(np.float32))

    XD=gpuarray.empty(NumG*NumD,dtype=np.int32)
    YD=gpuarray.empty(NumG*NumD,dtype=np.int32)
    OffsetD=gpuarray.empty(NumG*NumD,dtype=np.int32)

    MaskD=gpuarray.empty(NumG*NumD,dtype=np.bool_)
    TrueMaskD=gpuarray.empty(NumG*NumD,dtype=np.bool_)

    sim_func(XD,YD,OffsetD,MaskD,TrueMaskD,
            np.float32(x), np.float32(y),afDetInfoD,SD,
            whichOmegaD,np.int32(NumD),np.int32(NumG),np.float32(exp['energy']),np.int32(45),LimD,np.int32(5),
             block=(NumG,1,1),grid=(NumD,1))
    
    scoreD=gpuarray.empty(NumD,dtype=np.float32)

    BlockSize=256
    hit_func(scoreD,
            XD,YD,OffsetD,MaskD,TrueMaskD,
            MaxIntD,afDetInfoD,np.int32(NumG),np.int32(NumD),np.int32(45),
            block=(BlockSize,1,1),grid=(int(NumD/BlockSize+1),1))
    
    
    score=scoreD.get()
    args=np.argpartition(score,-numCut)[-numCut:]
    cov=np.cov(S[args].reshape((numCut,9),order='C').T)
    mean=np.mean(S[args],axis=0)
    for ii in range(MaxIter):
#         print np.max(score)
        S=np.random.multivariate_normal(
            np.zeros(9),cov,size=(NumD)).reshape((NumD,3,3),order='C')+np.tile(mean,(NumD,1,1))
        SD=gpuarray.to_gpu(S.ravel().astype(np.float32))

        XD=gpuarray.empty(NumG*NumD,dtype=np.int32)
        YD=gpuarray.empty(NumG*NumD,dtype=np.int32)
        OffsetD=gpuarray.empty(NumG*NumD,dtype=np.int32)

        MaskD=gpuarray.empty(NumG*NumD,dtype=np.bool_)
        TrueMaskD=gpuarray.empty(NumG*NumD,dtype=np.bool_)

        sim_func(XD,YD,OffsetD,MaskD,TrueMaskD,
                np.float32(x), np.float32(y),afDetInfoD,SD,
                whichOmegaD,np.int32(NumD),np.int32(NumG),np.float32(exp['energy']),np.int32(45),LimD,np.int32(5),
                 block=(NumG,1,1),grid=(NumD,1))

        scoreD=gpuarray.empty(NumD,dtype=np.float32)

        BlockSize=256
        hit_func(scoreD,
                XD,YD,OffsetD,MaskD,TrueMaskD,
                MaxIntD,afDetInfoD,np.int32(NumG),np.int32(NumD),np.int32(45),
                block=(BlockSize,1,1),grid=(int(NumD/BlockSize+1),1))


        score=scoreD.get()
        
        args=np.argpartition(score,-numCut)[-numCut:]
        cov=np.cov(S[args].reshape((numCut,9),order='C').T)
        mean=np.mean(S[args],axis=0)
        if np.trace(np.absolute(cov))<1e-9:
            break
        if np.min(score)==np.max(score):
            break

    return cov,mean,np.max(score)

In [28]:
dump,mean,score=CrossEntropyMethod(np.float32(-0.163125), np.float32(0.107171))
print(mean)
print(score)

[[  1.00900047e+00  -9.77232477e-05   1.21486647e-03]
 [  2.15229161e-04   1.00905543e+00   6.87461689e-04]
 [ -1.41546166e-03  -6.05867315e-04   1.00711550e+00]]
78.1311


In [ ]:
import numpy as np
from MicFileTool import read_mic_file
sw,snp=read_mic_file('/home/fyshen13/Strain/Reconstructions/Ti7_WithHRM_2ndLoad_z1_.mic.LBFS')
t=snp[:,6:9]-np.tile(np.array([298.089, 65.4218, 42.9553]),(snp.shape[0],1)) #g15Ps1_2nd
t=np.absolute(t)<1
t=t[:,0]*t[:,1]*t[:,2] #voxels that within 1 degree misorientation
t=snp[t]
x=t[:,0]
y=t[:,1]
con=t[:,9]

In [ ]:
tmpxx=x[con>0.7]
tmpyy=y[con>0.7]

AllMaxScore=[]
AllMaxS=[]
for ii in range(len(tmpxx)):
    if ii==0:
        t=CrossEntropyMethod(tmpxx[ii],tmpyy[ii])
        print ii,t[0]
        AllMaxScore.append(t[0])
        AllMaxS.append(t[1])
    else:
        t=CrossEntropyMethod(tmpxx[ii],tmpyy[ii],S_init=AllMaxS[-1])
        print ii,t[0]
        AllMaxScore.append(t[0])
        AllMaxS.append(t[1])

In [ ]:
from scipy.linalg import polar
realS=np.empty(AllMaxS.shape)
realO=np.empty(AllMaxS.shape)
for ii in range(len(realS)):
    #lattice constant I used in python nfHEDM scripts are different from the ffHEDM reconstruction used
    t=np.linalg.inv(AllMaxS[ii].T).dot(g15orien2nd).dot([[2.95/2.9254,0,0],[0,2.95/2.9254,0],[0,0,4.7152/4.674]])
    realO[ii],realS[ii]=polar(t,'left')